# Test Collisions
A visual test of a "rear-shunt" collision, to ensure that the agent does not get marked as collided permananently.

In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core import display 
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import PIL
from IPython import display
from ipycanvas import canvas
import time

In [3]:
from flatland.envs import malfunction_generators as malgen
from flatland.envs.agent_utils import EnvAgent
#from flatland.envs import sparse_rail_gen as spgen
from flatland.envs import rail_generators as rail_gen
from flatland.envs import agent_chains as ac
from flatland.envs.rail_env import RailEnv, RailEnvActions
from flatland.envs.persistence import RailEnvPersister
from flatland.utils.rendertools import RenderTool
from flatland.utils import env_edit_utils as eeu
from flatland.utils import jupyter_utils as ju

In [4]:
env, envModel = eeu.makeTestEnv("merging_spurs", nAg=2, bUCF=True)
behaviour = ju.ForwardWithPause(env, [ju.AgentPause(0, 10, 5)])
oEC = ju.EnvCanvas(env, behaviour)
env.reset(regenerate_rail=False)
oEC.show()
for i in range(25):
    oEC.step()
    oEC.render()
    time.sleep(0.1)

Canvas(layout=Layout(height='300px', width='600px'), size=(600, 300))

In [5]:
dAgStateExpected = {0: (7, 15, 2), 1: (6, 15, 2)}
dAgState={}
for iAg, ag in enumerate(env.agents):
    dAgState[iAg] = (*ag.position, ag.direction)
assert dAgState == dAgStateExpected